In [1]:
import joblib
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


vectorizer=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\TFIDF.sav")

decisiontree2c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\decisiontree2c.sav")
randomforest2c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\randomforest2c.sav")
bilstm2c=keras.models.load_model(r"C:\Users\ANSWEB\Desktop\Major Project\Code\bilstm2c.keras")

tokenizer2c=joblib.load(r"C:\Users\ANSWEB\Desktop\Major Project\Code\TOKENIZER2C.sav")

In [2]:
stop_words = set(stopwords.words('english'))
punctuation_pattern = '[!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~]'
lemmatizer = WordNetLemmatizer()

def senti_predict_2C(input_text):
    def text_clean(x) :
        # remove newline and carriage return
        x = x.replace('\n', ' ').replace('\r', '')
        # remove the links
        x = nltk.re.sub(r'http:\S+|https:\S+|www\S+', '', x)
        # remove punctuation marks and emoticons 
        x = nltk.re.sub(punctuation_pattern, '', x)
        words = x.split(' ')
        temp = []

        for word in words :
            word=word.lower()
            if (word not in stop_words) and (not word.isnumeric()) and (not word.startswith('@')):
                if word.startswith('#'):
                    temp.append(word[1:])
                else:
                    temp.append(word)

        return ' '.join(temp)


    def lemm(text):

        # Tokenize the text into words
        tokens = nltk.word_tokenize(text)

        # Lemmatize each word
        lemmatized_words = []
        for token in tokens:
            # Get the part-of-speech (POS) tag to improve lemmatization accuracy
            pos_tag = nltk.pos_tag([token])[0][1]
            pos = wordnet.VERB if pos_tag.startswith('V') else wordnet.NOUN if pos_tag.startswith('N') else wordnet.ADJ if pos_tag.startswith('J') else wordnet.ADV
            lemma = lemmatizer.lemmatize(token, pos=pos)
            lemmatized_words.append(lemma)

        # Join the lemmatized words back into a sentence
        lemmatized_text = ' '.join(lemmatized_words)

        return lemmatized_text

    text=text_clean(input_text)
    text=lemm(text)

    vectors = vectorizer.transform([text])
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    tk = tokenizer2c.texts_to_sequences(text)
    tk = pad_sequences(tk, maxlen=100, padding='post')
    dict={}
    
    dict['BiLSTM2C'] = bilstm2c.predict(tk)[0]
    
    dict['RandomForest2C'] = randomforest2c.predict(df)[0]

    dict['DecisionTree2C'] = decisiontree2c.predict(df)[0]
    
    return dict

In [6]:
test="covid is bad and dangerous"
category=['Negative','Positive']
print(senti_predict_2C, category[np.argmax(senti_predict_2C)])
print(senti_predict_2C(test))

<function senti_predict_2C at 0x000002029A595120> Negative
1/1 [==============================] - 0s 30ms/step
{'BiLSTM2C': array([0.9968641 , 0.00371791], dtype=float32), 'RandomForest2C': 'Negative', 'DecisionTree2C': 'Negative'}
